# Model comparison with PyMC3
### Brett Morris

EEG Meeting, 27 April 2021

# Setup
Install the necessary packages with: 

```bash
conda install -c conda-forge pymc3
pip install pymc3-ext corner arviz
```

**Troubleshooting**: Getting PyMC3 up and running can be complicated, see these pages for the most up-to-date notes on how to install on: [Mac](https://github.com/pymc-devs/pymc3/wiki/Installation-Guide-(MacOS)), [Linux](https://github.com/pymc-devs/pymc3/wiki/Installation-Guide-(Linux)), [Windows](https://github.com/pymc-devs/pymc3/wiki/Installation-Guide-(Windows)). If those instructions fail, you can usually get quick help on this [PyMC discourse page](https://discourse.pymc.io).

# Background

Often in observational astronomy, we have several plausible physical models which could fit the data, and we wish to identify which model best fits the data with the fewest free parameters. This is often the case with CHEOPS observations, where the observations can often be modeled as a linear combination of several *basis vectors*, or vectors of measurements like the telescope temperature or time, with the same length as your time series measurements $y$. **See also** [this repo](https://github.com/bmorris3/photometry_linalg) which I presented at a previous group meeting on photometric reductions with linear regression.

### Linear algebra recap

In a linear observation model, we often suppose that our observations $y(t)$ can be approximated with a linear combination of the $N$ basis vectors $a_i(t)$ like so: 

$$ y(t) = w_0 a_0(t) + w_1 a_1(t) + ... + w_N a_N(t)$$

where the $w_i$ coefficients are the *weights* on each basis vector which minimize the residuals between the observations and the linear combination of basis vectors. This is a least-squares problem, which can be phrased like so: the design matrix $X$,

\begin{equation}
{\bf X}  = \begin{bmatrix}
a_0(t_0) & a_1(t_0) & ... & a_N(t_0)\\
a_0(t_1) & a_1(t_1) & ... & a_N(t_1)\\
\vdots & \vdots & \vdots & \vdots \\
a_0(t_n) & a_1(t_n) & ... & a_N(t_n)\\
\end{bmatrix}
\end{equation}

is the concatenation of each basis column vector $a_i$. We want to solve for the weights 
\begin{equation}
{\bf \hat{\beta}} = \left[ w_0 ~ w_1 ~ ... ~  w_N \right]
\end{equation}
which minimize

$$ \arg \min_\beta || {\bf y - X \hat{\beta}} ||. $$

The best-fit estimators $\hat{\beta}$ can be solved with some matrix math (see [Wikipedia](https://en.wikipedia.org/wiki/Ordinary_least_squares) for more):

$${\displaystyle {\hat {\boldsymbol {\beta }}}=(\mathbf {X} ^{\rm {T}} N^{-1} \mathbf {X} )^{-1}\mathbf {X} ^{\rm {T}} N^{-1} \mathbf {y}, }$$

where $\bf N = \sigma_y^2 {\bf I}$ is the observational uncertainty on each measurement $\sigma_y$, multiplied by the identity matrix $\bf I$.

### Why use PyMC3?

> PyMC3 is a new, open-source [pure Python] framework with an intuitive and readable, yet powerful, syntax that is close to the natural syntax statisticians use to describe models. It features next-generation Markov chain Monte Carlo (MCMC) sampling algorithms such as the **No-U-Turn Sampler** (NUTS; Hoffman, 2014), a self-tuning variant of Hamiltonian Monte Carlo (HMC; Duane, 1987). This class of samplers works well on **high dimensional and complex posterior distributions** and allows many complex models to be fit without specialized knowledge about fitting algorithms.

> While most of PyMC3’s user-facing features are written in pure Python, it leverages Theano (Bergstra et al., 2010) to transparently **transcode models to C** and compile them to machine code, thereby boosting performance.

Source: [PyMC3 docs](https://docs.pymc.io/notebooks/getting_started.html), see also the [Example Gallery](https://docs.pymc.io/nb_examples/index.html), and [this manuscript on HMC methods](https://arxiv.org/abs/1701.02434)


# Example with [PyMC3](https://docs.pymc.io)
Next let's import the necessary packages:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import pymc3_ext as pmx
import arviz as az
from corner import corner

Generate an example dataset with three components: linear, quadratic, and sinusoidal trends. Add random noise:

In [ ]:
np.random.seed(42) 

n = 1000
x = np.linspace(-6, 6, n)

truths = [
    2,    # DC offset of the whole curve
    3,    # linear trend weight
    0.2,  # quadratic trend weight
    2     # sinusoidal weight
]

true_linear = truths[0] + truths[1] * x
true_period = 0.8
true_quadratic = truths[2] * x**2
true_sinusoid = truths[3] * np.sin(2 * np.pi * x / true_period)

uncertainties = 0.5
noise = np.random.normal(scale=0.5, size=n)

y = (
    true_linear + 
    true_quadratic + 
    true_sinusoid 
) + noise


plt.figure(figsize=(8, 4))
plt.plot(x, y, 'k.', label='observations')
plt.plot(x, true_linear, label='linear')
plt.plot(x, true_sinusoid, label='sin')
plt.plot(x, true_quadratic, label='quad')
plt.legend()
plt.gca().set(xlabel='$x$', ylabel='$y$')

Create a list of (unscaled) basis vectors, which we will use to solve for the linear regression coefficients (weights):

In [ ]:
basis_vectors = [
    np.ones_like(x), # DC offset
    x,               # linear trend
    x**2,            # quadratic trend
    np.sin(2 * np.pi * x / true_period), # sinusoid
     -(np.abs(x) < 2).astype(int) # extra, unnecessary vector    
]

basis_vector_names = [
    "unit", 
    "$x$", 
    "$x^2$", 
    "$\sin$",
    "box"
]

Loop through basis vectors, adding one at a time, and evaluating the best-fit linear regression coefficients for each combination of basis vectors. Save results to a dictionary called ``inference_data_dict``, and plot the *maximum a posteriori* (MAP) solution in red over the data in black:

In [ ]:
total_basis_vectors = len(basis_vectors)

inference_data_dict = {}

for n_basis_vectors in range(1, total_basis_vectors + 1):
    
    model_name = f"n={n_basis_vectors}"
    
    with pm.Model(name=model_name) as pymc3_model: 
    
        beta = pm.Uniform("beta", lower=-10, upper=10, shape=n_basis_vectors)
    
        linear_model = pm.math.dot(beta, basis_vectors[:n_basis_vectors])
    
        pm.Normal("obs", mu=linear_model, sigma=uncertainties, observed=y)
        
        map_soln = pm.find_MAP()

        plt.plot(x, y, '.k')
        plt.plot(x, pmx.eval_in_model(linear_model, map_soln), 'r')
        plt.title("Basis vectors: " + ", ".join(basis_vector_names[:n_basis_vectors]))
        plt.gca().set(xlabel='$x$', ylabel='$y$')
        plt.show()
        
        inference_data = pm.sample(
            draws=1000,
            tune=1000,
            compute_convergence_checks=False,
            return_inferencedata=True,
            target_accept=0.9,
            cores=2
        )
        
        inference_data_dict[model_name] = inference_data

## Model comparison

Now that we've solved for the best-fit weights on each basis vector for all five basis vectors. Which combination of basis vectors best reproduces the observations? **Which model do we choose?**

One technique for measuring the relative confidence that a set of models fits the data is the [Leave-One-Out cross validation statistic](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#Leave-one-out_cross-validation). In its simplest form, the LOO will do the following: 

1. Divide the observations into test and training groups of length $1$ and $n-1$
2. Fit the model on the $n-1$ training observations, predict measurement at the "missing" data point
3. Compare the prediction at the "missing" point to the actual observation
4. Sum the squares of these residuals and normalize by the number of data points

In [ ]:
comparison_loo = az.compare(
    inference_data_dict, 
    ic='loo'
)

comparison_loo

For illustrative purposes, we compute the leave-one-out cross validation statistic in the [style of the wikipedia entry](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#Leave-one-out_cross-validation) for each model explicitly here:

In [ ]:
# Loop over the number of basis vectors:
for n_basis_vectors in range(1, len(basis_vectors) + 1):
    
    # Construct a design matrix out of the first N basis vectors
    design_matrix = np.vstack(basis_vectors[:n_basis_vectors]).T

    # Compute the LOO by leaving out one data point, refitting the
    # model, and subtracting the best-fit expectation from the 
    # actual observation at the missing data point:
    loo_statistic = 0
    for i in range(len(x)):
        indices = np.r_[0:i-1, i+1:len(x)]   # indexing trick to skip `i`th index
        x_in = x[indices]
        y_in = y[indices]
        design_matrix_in = design_matrix[indices, :]
        
        # Compute the least squares regression:
        betas = np.linalg.lstsq(design_matrix_in, y_in, rcond=-1)[0]
        
        # Compute the value of the model at the missing time
        y_out = design_matrix[i, :] @ betas

        # Compute the difference between expected and observed at
        # the missing data point:
        loo_statistic += (y[i] - y_out)**2
        
    # Normalize over the length of the dataset:
    loo_statistic /= len(x)
    print(f"LOO for n={n_basis_vectors}: {loo_statistic}")

Note that the LOO is smallest for the $n=\{4,5\}$ models, and in that case, you may want to select the model with the fewest free parameters, in this case $n=4$.

We can also compute other information criteria, such as the "widely applicable information criterion" (read [all of the details here](http://www.stat.columbia.edu/~gelman/research/published/waic_understand3.pdf)) with arviz: 

In [ ]:
comparison_waic = az.compare(
    inference_data_dict, 
    ic='waic'
)

comparison_waic

If you compare the WAIC stats to the LOO stats above, you'll see that they agree closely.

Finally, let's make a corner plot of the $n=4$ model to see how well the best-fit predictions for ${\bf \hat{\beta}}$ compare with the true (input) values:

In [ ]:
# Convert the posteriors to a flat chain for plotting a corner plot:
post = inference_data_dict['n=4'].posterior['n=4_beta'].values
flatchain = post.reshape((post.shape[0] * post.shape[1], -1))

In [ ]:
corner(flatchain, labels=basis_vector_names, truths=truths);

The agreement is quite good!